# Logging

The [logging](https://docs.python.org/3/library/logging.html#module-logging) module is part of the Python standard library. It is described in [PEP 0282](https://www.python.org/dev/peps/pep-0282). You can get a first introduction to the module in the [Basic Logging Tutorial](https://docs.python.org/3/howto/logging.html#logging-basic-tutorial).

Alternatively, you can also use [loguru](https://github.com/Delgan/loguru), which makes logging almost as easy as using `print` instructions.

Logging usually serves two different purposes:

**Diagnosis:**

* You can display the context of certain events.
* Tools like [Sentry](https://sentry.io/) group related events and facilitate user identification, etc., so that developers can find the cause of the error more quickly.

**Monitoring:** 

* The logging records events for user-defined heuristics, e.g. for business analyses. These records can be used for reports or optimisation of the business goals and, if necessary, visualised.

What are the advantages of `logging` over `print`?

* The log file contains all available diagnostic information such as file name, path, function and line number.
* All events are automatically available via the root logger unless they are explicitly filtered out.
* Logging can be muted using either of the following two methods: [logging.Logger.setLevel()](https://docs.python.org/3/library/logging.html#logging.Logger.setLevel) or [logging.disabled](https://docs.python.org/3/library/logging.html#logging.disable).

In [1]:
import logging

## Example configuration via an INI file

The following example loads the file `development.ini` in this directory:

```ini
[loggers]
keys=root

[handlers]
keys=stream_handler

[formatters]
keys=formatter

[logger_root]
level=DEBUG
handlers=stream_handler

[handler_stream_handler]
class=StreamHandler
level=DEBUG
formatter=formatter
args=(sys.stderr,)

[formatter_formatter]
format=%(asctime)s %(name)-12s %(levelname)-8s %(message)s
```

In [2]:
from logging.config import fileConfig

In [3]:
fileConfig('development.ini')
d = {'clientip': '192.168.0.1', 'user': 'fbloggs'}
logger = logging.getLogger('tcpserver')
logger.warning('Protocol problem: %s', 'connection reset', extra=d)

2019-10-21 16:25:30,681 tcpserver    WARNING  Protocol problem: connection reset


**Pro:**

* Possibility to update the configuration on the fly by using the function `logging.config.listen()` to listen on a socket.
* Different configurations can be used in different environments, for example in `development.ini` the log level can be specified as `DEBUG` while the log level in `production.ini` used `WARN`.

**Con:**

* Less control, for example, compared to user-defined filters or loggers that are configured in the code.

## Example for a configuration via a dictionary

In [4]:
import logging
from logging.config import dictConfig

logging_config = dict(
    version = 1,
    formatters = {
        'f': {'format':
              '%(asctime)s %(name)-12s %(levelname)-8s %(message)s'}
        },
    handlers = {
        'h': {'class': 'logging.StreamHandler',
              'formatter': 'f',
              'level': logging.DEBUG}
        },
    root = {
        'handlers': ['h'],
        'level': logging.DEBUG,
        },
)

dictConfig(logging_config)

**Pro:**

* Update during operation

**Con:**

* Less control than when configuring a logger in code

## Example configuration directly in the code

In [5]:
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

Alternatively, you can also use *Magic Commands*:

| Befehl         | Beschreibung                                                                              |
| -------------- | ----------------------------------------------------------------------------------------- |
| `%logstart`    | Starts logging anywhere in a session                                                      |
|                | `%logstart [-o\|-r\|-t\|-q] [log_name [log_mode]]`                                        |
|                | If no name is given, `ipython_log.py` is used in the current directory.                   |
|                | `log_mode` is an optional parameter. The following modes can be specified:                |
|                | – `append` appends the logging information to the end of an existing file                 |
|                | – `backup` renames the existing file to `name~` and writes to `name`                      |
|                | – `global` appends the logging information at the end of an existing file                 |
|                | – `over` overwrites an existing log file                                                  |
|                | – `rotate` creates rotating log files: `name.1~`, `name.2~`, etc.                         |
|                | Options:                                                                                  |
|                | – `-o` also logs the output of IPython                                                    |
|                | – `-r` logs raw output                                                                    |
|                | – `-t` writes a time stamp in front of each log entry                                     |
|                | – `-q` suppresses the logging output                                                      |
| `%logon`       | Restart the logging                                                                       |
| `%logoff`      | Temporary termination of logging                                                          |

**Pro:**

* Complete control over the configuration

**Con:**

* Changes in the configuration require a change in the source code

> **See also:**
> 
> * [logging configuration](https://docs.python.org/3/howto/logging.html#configuring-logging)
